Data prprocessing and loading

In [3]:
import pandas as pd
import cv2
import numpy as np

def load_images_and_labels(csv_file):
    # Load CSV file containing image pair paths and labels
    data = pd.read_csv(csv_file)

    # Initialize lists to store images and labels
    images = []
    labels = []

    # Iterate through each row in the CSV file
    for index, row in data.iterrows():
        # Read the pair of images
        image1 = cv2.imread(f"Dataset/Training_frame_pairs/{row['Pair']}/frame_1.jpg")
        image2 = cv2.imread(f"Dataset/Training_frame_pairs/{row['Pair']}/frame_2.jpg")
        
        # Check if images are successfully loaded
        if image1 is not None and image2 is not None:
            # Preprocess images (e.g., resize, normalize pixel values)
            image1 = preprocess_image(image1)
            image2 = preprocess_image(image2)
            
            # Combine the two images into one array (if needed)
            combined_image = np.concatenate((image1, image2), axis=1)
            
            # Append the combined image to the list of images
            images.append(combined_image)
            
            # Append the label to the list of labels
            labels.append(row['Direction'])
        else:
            print(f"Failed to load image pair: {row['Pair']}")

    return np.array(images), np.array(labels)

def preprocess_image(image, target_size=(224, 224)):
    # Resize image to target dimensions
    image = cv2.resize(image, target_size)
    
    # Normalize pixel values (if needed)
    image = image.astype(np.float32) / 255.0
    
    return image

# Load and preprocess training data
train_images, train_labels = load_images_and_labels('Dataset/Training_data_direction_labels.csv')

# Load and preprocess test data
test_images, test_labels = load_images_and_labels('Dataset/Test_data_direction_labels.csv')

# Print the shapes of the loaded data
print("Training data shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test data shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Training data shape: (2223, 224, 448, 3)
Training labels shape: (2223,)
Test data shape: (901, 224, 448, 3)
Test labels shape: (901,)


Model architectur and training

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 448, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Assuming 7 output classes
])
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Convert string labels to integer labels
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using encoded labels
history = model.fit(train_images, train_labels_encoded, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 675s 11s/step - accuracy: 0.7236 - loss: 1.2172 - val_accuracy: 0.5124 - val_loss: 1.0673
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 770s 13s/step - accuracy: 0.9853 - loss: 0.0417 - val_accuracy: 0.5640 - val_loss: 1.2673
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 550s 9s/step - accuracy: 0.9867 - loss: 0.0439 - val_accuracy: 0.5663 - val_loss: 1.3500
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 642s 11s/step - accuracy: 0.9972 - loss: 0.0146 - val_accuracy: 0.4607 - val_loss: 2.2490
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 358s 6s/step - accuracy: 0.9899 - loss: 0.0466 - val_accuracy: 0.6090 - val_loss: 1.4027
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 328s 6s/step - accuracy: 0.9959 - loss: 0.0078 - val_accuracy: 0.4382 - val_loss: 3.0206
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 182s 3s/step - accuracy: 0.9967 - loss: 0.0073 - val_accuracy: 0.6337 - val_loss: 1.2215
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 363s 6s/step - accuracy: 0.9975 - loss: 0.0073 - val_accuracy: 0.4562 